# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import the cities _output.csv file from Part 1
cities_df = pd.read_csv('../output_data/cities_output.csv')
cities_df

,Unnamed: 0,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,0,GL,qaanaaq,77.4840,-69.3632,1643498180,100,64,4.95,10.40
1,1,AU,hobart,-42.8794,147.3294,1643498725,75,60,72.01,3.00
2,2,GL,qasigiannguit,68.8193,-51.1922,1643498725,93,81,12.45,12.71
3,3,IN,amalapuram,16.5833,82.0167,1643498725,8,95,62.76,1.86
4,4,AU,carnarvon,-24.8667,113.6333,1643498726,67,88,77.07,12.66
...,...,...,...,...,...,...,...,...,...,...
544,544,JP,naze,28.3667,129.4833,1643499473,100,76,56.32,12.55
545,545,JP,itoman,26.1247,127.6694,1643499566,75,92,64.42,13.80
546,546,RU,solnechnyy,50.7214,136.6319,1643499626,78,89,-10.21,0.81
547,547,BD,sarankhola,22.3082,89.7897,1643499627,0,56,53.76,2.48


In [3]:
#drop the duplicate index column created during the csv process in Part !
cities_df.drop('Unnamed: 0', inplace=True, axis=1)


In [4]:
display(cities_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,GL,qaanaaq,77.4840,-69.3632,1643498180,100,64,4.95,10.40
1,AU,hobart,-42.8794,147.3294,1643498725,75,60,72.01,3.00
2,GL,qasigiannguit,68.8193,-51.1922,1643498725,93,81,12.45,12.71
3,IN,amalapuram,16.5833,82.0167,1643498725,8,95,62.76,1.86
4,AU,carnarvon,-24.8667,113.6333,1643498726,67,88,77.07,12.66
...,...,...,...,...,...,...,...,...,...
544,JP,naze,28.3667,129.4833,1643499473,100,76,56.32,12.55
545,JP,itoman,26.1247,127.6694,1643499566,75,92,64.42,13.80
546,RU,solnechnyy,50.7214,136.6319,1643499626,78,89,-10.21,0.81
547,BD,sarankhola,22.3082,89.7897,1643499627,0,56,53.76,2.48


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
humidity = cities_df["humidity"].astype(float)
maxhumidity = cities_df["humidity"].astype(float).max()
locs_df = cities_df[["lat", "lon"]]
locs_df

,lat,lon
0,77.4840,-69.3632
1,-42.8794,147.3294
2,68.8193,-51.1922
3,16.5833,82.0167
4,-24.8667,113.6333
...,...,...
544,28.3667,129.4833
545,26.1247,127.6694
546,50.7214,136.6319
547,22.3082,89.7897


In [6]:
maxhumidity

100.0

In [7]:
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
#Best website for explaining how to use google maps
#Chose the center based on a value in dataframe--lat and lon
gmaps.configure(api_key = g_key)
figure = gmaps.figure(zoom_level=3,center=(48.8566,2.3522))
heat_layer = gmaps.heatmap_layer(locs_df, weights=humidity,dissipating=False,max_intensity=maxhumidity,point_radius=2)
figure.add_layer(heat_layer)
figure



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cities_df

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,GL,qaanaaq,77.4840,-69.3632,1643498180,100,64,4.95,10.40
1,AU,hobart,-42.8794,147.3294,1643498725,75,60,72.01,3.00
2,GL,qasigiannguit,68.8193,-51.1922,1643498725,93,81,12.45,12.71
3,IN,amalapuram,16.5833,82.0167,1643498725,8,95,62.76,1.86
4,AU,carnarvon,-24.8667,113.6333,1643498726,67,88,77.07,12.66
...,...,...,...,...,...,...,...,...,...
544,JP,naze,28.3667,129.4833,1643499473,100,76,56.32,12.55
545,JP,itoman,26.1247,127.6694,1643499566,75,92,64.42,13.80
546,RU,solnechnyy,50.7214,136.6319,1643499626,78,89,-10.21,0.81
547,BD,sarankhola,22.3082,89.7897,1643499627,0,56,53.76,2.48


In [20]:
perfect_df = cities_df.loc[
    (cities_df['max_temp'] > 66) &
    (cities_df['max_temp'] < 74) &
    (cities_df['wind_speed']<=10) &
    (cities_df['humidity']<=50)].reset_index(drop=True).dropna()

display(perfect_df)



,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,NZ,waipawa,-41.4122,175.5153,1643498588,42,50,70.43,7.20
1,US,coronado,32.6859,-117.1831,1643498737,75,22,70.63,3.44
2,AU,new norfolk,-42.7826,147.0587,1643498804,16,36,72.66,2.06
3,US,laguna,38.4210,-121.4238,1643498884,0,42,66.96,3.00
4,US,half moon bay,37.4636,-122.4286,1643499035,0,47,67.91,1.01
5,MR,atar,20.5169,-13.0499,1643499066,100,21,73.76,5.21
6,TD,pala,9.3642,14.9046,1643499328,72,16,69.66,9.89
7,US,altus,34.6381,-99.3340,1643499336,0,13,68.07,8.05
8,NZ,waiouru,-39.4833,175.6667,1643499403,0,27,73.78,5.70
9,US,salinas,36.6777,-121.6555,1643499242,0,38,69.89,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = perfect_df

hotel_df['Hotel Name'] = ''

display(hotel_df.head())

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,NZ,waipawa,-41.4122,175.5153,1643498588,42,50,70.43,7.20,
1,US,coronado,32.6859,-117.1831,1643498737,75,22,70.63,3.44,
2,AU,new norfolk,-42.7826,147.0587,1643498804,16,36,72.66,2.06,
3,US,laguna,38.4210,-121.4238,1643498884,0,42,66.96,3.00,
4,US,half moon bay,37.4636,-122.4286,1643499035,0,47,67.91,1.01,


In [22]:
radius = 5000
place = 'hotel'

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    
    query_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C{lon}&radius={radius}&type={place}&key={g_key}"

    result = requests.get(query_url).json()
    
    try:
        hotel_name = result['results'][1]['name']
    except:
        hotel_name = 'NA'
    
    hotel_df.at[index, 'Hotel Name'] = hotel_name


In [23]:
display(hotel_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,NZ,waipawa,-41.4122,175.5153,1643498588,42,50,70.43,7.20,Tuturumuri School
1,US,coronado,32.6859,-117.1831,1643498737,75,22,70.63,3.44,El Cordova Hotel
2,AU,new norfolk,-42.7826,147.0587,1643498804,16,36,72.66,2.06,The Shingles Riverside Cottages
3,US,laguna,38.4210,-121.4238,1643498884,0,42,66.96,3.00,Holiday Inn Express & Suites Elk Grove Central...
4,US,half moon bay,37.4636,-122.4286,1643499035,0,47,67.91,1.01,"The Ritz-Carlton, Half Moon Bay"
5,MR,atar,20.5169,-13.0499,1643499066,100,21,73.76,5.21,Oumou Elghoura
6,TD,pala,9.3642,14.9046,1643499328,72,16,69.66,9.89,Pala Urbain Centre de Santé
7,US,altus,34.6381,-99.3340,1643499336,0,13,68.07,8.05,"Holiday Inn Express & Suites Altus, an IHG Hotel"
8,NZ,waiouru,-39.4833,175.6667,1643499403,0,27,73.78,5.70,National Army Museum
9,US,salinas,36.6777,-121.6555,1643499242,0,38,69.89,4.61,Laurel Inn & Conference Center


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [28]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))